In [1]:
import glob
import json
import datetime
from dateutil import parser

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML, Image, Video

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-pastel')
#plt.style.use('style.mplstyle')

#!brew install ffmpeg      # MPE4 animations
#!brew install imagemagick  # Gif animations

In [2]:
with open('gitignore/data.json', 'r') as file:
    buy_and_sell_curves = json.load(file)
    for buy_and_sell_curve in buy_and_sell_curves:
        buy_and_sell_curve["date_time"] = parser.parse(buy_and_sell_curve["date_time"], yearfirst=True)
        buy_and_sell_curve["buy_curve"] = pd.DataFrame(buy_and_sell_curve["buy_curve"], columns=["Price", "Volume"])
        buy_and_sell_curve["sell_curve"] = pd.DataFrame(buy_and_sell_curve["sell_curve"], columns=["Price", "Volume"])
#buy_and_sell_curves

In [6]:
class Plot():
    def __init__(
                self, 
                buy_and_sell_curves,
                title,
                title2,
                source,
                draw_past_interactions=True, 
                dpi=200, 
                points_back_in_time = 24*7
            ):
        self.buy_and_sell_curves = buy_and_sell_curves
        self.draw_past_interactions = draw_past_interactions
        self.points_back_in_time = points_back_in_time
        
        # Variables
        self.fig = None
        self.ax = None
        self.line1 = None
        self.line2 = None
        self.line3 = None
        self.intersections = None
        self.past_intersections = None

        self.text_date = None
        self.text_price = None
        self.text_volume = None
        self.text_imported = None

        self.past_interacton_volume = None
        self.past_interacton_price = None
        
        #plt.rcParams['savefig.pad_inches'] = 0.4
        plt.rcParams["font.family"] = "Gill Sans"
        plt.rcParams["figure.facecolor"] = "ffffff"
        plt.rcParams["axes.facecolor"] = "ffffff"
        
        self.fig = plt.figure(figsize=(6.4, 3.6), dpi=dpi)
        self.ax = plt.axes(xlim=(0, 80000), ylim=(-20, 100))
        
        
        plt.margins(0.4, tight=False)

        self.ax.spines['top'].set_visible(False)
        self.ax.spines['right'].set_visible(False)
        self.ax.spines['bottom'].set_visible(True)
        self.ax.spines['left'].set_visible(True)
        
        self.ax.axhline(y=0, color="lightgray", lw=0.5)

        self.fig.text(0.05, 0.90, title, fontsize="x-large", weight="bold")
        self.fig.text(0.05, 0.85, title2)
        self.fig.text(0.90, 0.05, source, ha="right")
        plt.subplots_adjust(top=0.75, bottom=0.2)

        plt.xlabel("Volume (MW)")
        plt.ylabel("Electricity price (EUR/MWh)")

        self.text_date = self.ax.text(78000, 96, "", ha="right")
        self.text_price = self.ax.text(78000, 86, "", ha="right")
        self.text_volume = self.ax.text(78000, 76, "", ha="right")
        self.text_imported = self.ax.text(78000, 66, "", ha="right")

        self.line1, = self.ax.plot([], [], lw=3)
        self.line2, = self.ax.plot([], [], lw=1, color = '#94C7FD')
        self.line3, = self.ax.plot([], [], lw=3)
        
        self.past_interacton_volume = []
        self.past_interacton_price = []
        self.past_intersections, = self.ax.plot([], [], 'o', alpha=0.1, markersize=3)
        self.intersections, = self.ax.plot([], [], 'o')
        
        self.line1.set_label('Buy curve')
        self.line2.set_label('/w ex-/import')
        self.line3.set_label('Sell curve')
        self.ax.legend(loc='lower right', frameon=False)

    def init(self):
        self.line1.set_data([], [])
        self.line2.set_data([], [])
        self.line3.set_data([], [])
        self.past_intersections.set_data([], [])
        self.intersections.set_data([], [])

        return self.line1, self.line2, self.intersections,

    def animate(self, i):
        date_time = self.buy_and_sell_curves[i]["date_time"]
        buy_curve = self.buy_and_sell_curves[i]["buy_curve"]
        sell_curve = self.buy_and_sell_curves[i]["sell_curve"]
        
        imported_volume = self.buy_and_sell_curves[i]["imported_volume_volume"]
        accepted_buy_volume = self.buy_and_sell_curves[i]["accepted_buy_volume"]
        accepted_sell_volume = self.buy_and_sell_curves[i]["accepted_sell_volume"]
        volume = self.buy_and_sell_curves[i]["volume"]
        price = self.buy_and_sell_curves[i]["price"]

        date_time_str = date_time.strftime("%Y-%m-%d %H:%M")
        self.text_date.set_text(date_time_str)
        self.text_price.set_text(" %4.1f EUR/MWh" % price)
        self.text_volume.set_text("%d MW produced" % round(volume, -3))
        self.text_imported.set_text("%d MW exported" % round(-1 * imported_volume, -2))

        self.line1.set_data(buy_curve["Volume"] + imported_volume, buy_curve["Price"])
        self.line2.set_data(buy_curve["Volume"], buy_curve["Price"])
        self.line3.set_data(sell_curve["Volume"], sell_curve["Price"])

        if self.draw_past_interactions:
            self.past_intersections.set_data(self.past_interacton_volume[-1*self.points_back_in_time:], self.past_interacton_price[-1*self.points_back_in_time:])
            self.past_interacton_volume.append(volume)
            self.past_interacton_price.append(price)

        self.intersections.set_data(volume, price)

        return self.line1, self.line2, self.intersections,
        
def save_image(buy_and_sell_curves, path, i, title, title2, source, dpi=100):
    plot = Plot(buy_and_sell_curves, title, title2, source, dpi=dpi)
    plot.animate(i);
    plot.fig.savefig(path, dpi=200)

def save_and_render_video(buy_and_sell_curves, path, title, title2, source, draw_past_interactions=False, points_back_in_time=7*24):
    plot = Plot(buy_and_sell_curves, title, title2, source, draw_past_interactions, points_back_in_time=points_back_in_time)
    frames = len(buy_and_sell_curves)
    anim = animation.FuncAnimation(plot.fig, plot.animate, init_func=plot.init, frames=frames, blit=True)
    anim.save(path, writer = animation.FFMpegWriter(fps = 10, codec='h264'))
    plt.close()
    
def save_and_render_gif(buy_and_sell_curves, path, title, title2, source, draw_past_interactions=False, points_back_in_time=7*24):
    plot = Plot(buy_and_sell_curves, title, title2, source, draw_past_interactions, points_back_in_time=points_back_in_time)
    frames = len(buy_and_sell_curves)
    anim = animation.FuncAnimation(plot.fig, plot.animate, init_func=plot.init, frames=frames, blit=True)
    anim.save(path, writer = animation.ImageMagickWriter(fps = 10, codec='h264'))
    plt.close()

## Render animations

In [8]:
path = "Bid curves on Nord Pool, 2019.mp4"
title="Supply and demand"
title2="Bid curves on Nord Pool, Jan–Dec 2019"
source="Jon Tingvold"
from_date = datetime.datetime(2019, 1, 1, 0, 0)
to_date = datetime.datetime(2019, 12, 31, 23, 0)

buy_and_sell_curve_for_plot = [
    buy_and_sell_curve for buy_and_sell_curve in buy_and_sell_curves 
        if  buy_and_sell_curve["date_time"] >= from_date
       and  buy_and_sell_curve["date_time"] <= to_date
]


save_and_render_video(
    buy_and_sell_curve_for_plot, 
    path, 
    title, 
    title2,
    source,
    draw_past_interactions=True, 
    points_back_in_time=7*24
)

HTML("""<video src="{}" width="640" height="360" controls="true">""".format(path))